In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

## Wczytanie danych

In [2]:
df = pd.read_hdf("../data/car.h5")
df.sample()

/home/adrian/miniconda3/envs/dataworkshop/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,feature_wspomaganie-kierownicy,feature_podgrzewana-przednia-szyba,feature_przyciemniane-szyby,feature_elektrycznie-ustawiane-fotele,feature_klimatyzacja-czterostrefowa,feature_tuner-tv,feature_poduszki-boczne-przednie,feature_tempomat-aktywny,feature_klimatyzacja-automatyczna,feature_poduszki-boczne-tylne,feature_radio-fabryczne,feature_odtwarzacz-dvd,feature_czujniki-parkowania-tylne,feature_łopatki-zmiany-biegów,feature_centralny-zamek,feature_dach-panoramiczny,feature_kamera-cofania,feature_immobilizer,feature_czujnik-martwego-pola,feature_gniazdo-sd,feature_podgrzewane-lusterka-boczne,feature_podgrzewane-przednie-siedzenia,feature_hak,feature_abs,feature_wielofunkcyjna-kierownica,feature_światła-xenonowe,feature_gniazdo-usb,feature_tapicerka-skórzana,feature_gniazdo-aux,feature_esp-(stabilizacja-toru-jazdy),feature_alarm,feature_podgrzewane-tylne-siedzenia,feature_mp3,feature_tempomat,feature_hud-(wyświetlacz-przezierny),feature_elektrochromatyczne-lusterko-wsteczne,feature_relingi-dachowe,feature_elektrochromatyczne-lusterka-boczne,param_liczba-miejsc,param_uszkodzony,param_marka-pojazdu,param_model,param_engine-code,param_liczba-drzwi,param_first-registration,param_country-of-origin,param_pierwsza-rejestracja,param_gearbox,param_przebieg,param_faktura-vat,param_rok-produkcji,param_vin,param_perłowy,param_homologacja-ciężarowa,param_service-record,param_metallic,param_leasing-concession,param_color,param_financing-option,param_original-owner,param_vat-marża,param_kategoria,param_co2-emissions,param_leasing,param_mileage,param_zarejestrowany-jako-zabytek,param_napęd,param_wartość-wykupu,param_typ,param_metalik,param_pierwszy-właściciel,param_skrzynia-biegów,param_vat-discount,param_no-accident,param_liczba-pozostałych-rat,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
96832,"[Osobowe, Volkswagen, New Beetle]","17:22, 2 marca 2018",PLN,Cena Brutto,49994.0,"Dziećmorowice, wałbrzyski, Dolnośląskie",None,Osoba prywatna,False,False,False,False,False,False,True,True,True,True,True,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,True,True,False,True,False,False,False,True,

In [3]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
    if isinstance(df[feat][0], list):
        continue
    
    factorized_values = df[feat].factorize()[0]
    if SUFFIX_CAT in feat:
        df[feat] = factorized_values
    else:
        df[feat+ SUFFIX_CAT] = factorized_values
        
cat_feats =  [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) =='None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) =='None' else int(x.split('cm')[0].replace(' ', '')))


In [5]:
feats = [
    'param_rok-produkcji',
    'param_stan__cat',
    'param_napęd__cat',
    'param_skrzynia-biegów__cat',
    'param_moc',
    'param_faktura-vat__cat',
    'param_marka-pojazdu__cat',
    'param_typ__cat', 
    'feature_kamera-cofania__cat',
    'param_wersja__cat',
    'param_model-pojazdu__cat',
    'param_pojemność-skokowa',
    'param_kod-silnika__cat',
    'seller_name__cat',
    'feature_wspomaganie-kierownicy__cat',
    'feature_czujniki-parkowania-przednie__cat',
    'param_uszkodzony__cat',
    'feature_system-start-stop__cat',
    'feature_regulowane-zawieszenie__cat',
    'feature_asystent-pasa-ruchu__cat',
]

In [7]:
def run_model(model, feats):
    X = df[feats].values
    y = df['price_value'].values

    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
    return np.mean(scores), np.std(scores)

## XGBoost

In [9]:
xgb_params = {
    'max_depth':5,
    'n_estimatords':50,
    'learning_rate':0.1,
    'seed':0,
    'nthread': 3   
}

In [10]:
model =  xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[18:17:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:17:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:17:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8580.61650667904, 58.180956552556225)

In [14]:
def obj_func(params):
    print("Traniang with params: ")
    print(params)
    
    mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
    
    return {"loss": np.abs(mean_mae), "status": STATUS_OK}


xgb_reg_params = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror',
    'n_estimatords': 100,
    'seed':0,
    'nthread': 4 
}

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=30)

best

Traniang with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.3, 'max_depth': 5, 'n_estimatords': 100, 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Traniang with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.2, 'max_depth': 9, 'n_estimatords': 100, 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Traniang with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.25, 'max_depth': 7, 'n_estimatords': 100, 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Traniang with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.15000000000000002, 'max_depth': 15, 'n_estimatords': 100, 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Traniang with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 9, 'n_estimatords': 100, 'nthread': 4, 'objective': 'reg:squarederror', 'seed': 0,

{'colsample_bytree': 0.75,
 'learning_rate': 1,
 'max_depth': 10,
 'subsample': 1.0}